In [9]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# test the connection
# response = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[{"role": "user", "content": "Hello, how are you?"}]
# )

# print(response.choices[0].message.content)



In [6]:
from scraper import fetch_website_contents

content = fetch_website_contents("https://github.com/waydabber/BetterDisplay?tab=readme-ov-file")

print(content)

GitHub - waydabber/BetterDisplay: Unlock your displays on your Mac! Flexible HiDPI scaling, XDR/HDR extra brightness, virtual screens, DDC control, extra dimming, PIP/streaming, EDID override and lots more!

Skip to content
Navigation Menu
Toggle navigation
Sign in
Appearance settings
Platform
AI CODE CREATION
GitHub Copilot
Write better code with AI
GitHub Spark
Build and deploy intelligent apps
GitHub Models
Manage and compare prompts
MCP Registry
New
Integrate external tools
DEVELOPER WORKFLOWS
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
APPLICATION SECURITY
GitHub Advanced Security
Find and fix vulnerabilities
Code security
Secure your code as you build
Secret protection
Stop leaks before they start
EXPLORE
Why GitHub
Documentation
Blog
Changelog
Marketplace
View all features
Solutions
BY COMPANY SIZE
Enterprises
Small and medium teams
Startups
Nonprofits
BY USE CASE
App Modernization
DevSecOps
DevOps


In [ ]:

system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

def stream_gpt(prompt):
    text_resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        stream=True
    )
    
    final_text = ""

    for chunk in text_resp:
        if chunk.choices[0].delta.content is not None:
            final_text += chunk.choices[0].delta.content
            yield final_text

def stream_claude(prompt):
    text_resp = client.chat.completions.create(
        model="claude-3-haiku-20240307",
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        stream=True
    )

    final_text = ""

    for chunk in text_resp:
        if chunk.choices[0].delta.content is not None:
            final_text += chunk.choices[0].delta.content
            yield final_text

def stream_with_model(prompt, model):
    if model == "GPT":
        return stream_gpt(prompt)
    elif model == "Claude":
        return stream_claude(prompt)
    else:
        raise ValueError("Invalid model")


def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    
    resp = stream_with_model(prompt, model)
    yield from resp


# using gradio to create a web interface
import gradio as gr

gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Company name"), gr.Textbox(label="URL"), gr.Dropdown(choices=["GPT", "Claude"], value="GPT")],
    outputs=gr.Markdown(label="Brochure"),
    title="Brochure Generator",
    description="Generate a brochure for a company based on their landing page.",
    examples=[
        ["Hugging Face", "https://huggingface.co"],
        ["Edward Donner", "https://edwarddonner.com"]
    ]
).launch()


* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/thong.pham/Documents/thong/llm-engineering/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thong.pham/Documents/thong/llm-engineering/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thong.pham/Documents/thong/llm-engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thong.pham/Documents/thong/llm-engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/U